In [3]:
#Loading data
import numpy as np
import pandas as pd
import math
X_data = pd.read_csv('NURSERY.csv', names=["parents", "has_nurs", "form", "children", "housing", "finance", "social", "health", "class"])
shuffle_index=np.random.permutation(len(X_data))
X_data=X_data.iloc[shuffle_index,:]
X_data = X_data.reset_index(drop=True)

In [5]:
#Bootstrap Resampling
def bootsrap(X):
    n=len(X)
    index=np.floor(np.random.rand(n)*n).astype(int)
    p=int((2/3)*n)
    X_train=X.iloc[index[0:p],:-1].reset_index(drop=True)
    Y_train=X.iloc[index[0:p],-1].reset_index(drop=True)
    X_test=X.iloc[index[p:],:-1].reset_index(drop=True)
    Y_test=X.iloc[index[p:],-1].reset_index(drop=True)
    return X_train,Y_train,X_test,Y_test

In [18]:
#getting bootstrap data
X_train,Y_train,X_test,Y_test=bootsrap(X_data)

In [6]:
#Creating k_folds splits
def cross_validation_split(dataset, folds):
    fold_size = int(len(dataset) / folds)
    dataset_split = [[] for j in range(folds+1)]
    p=0
    i=0
    while i<len(dataset):
        fold = pd.DataFrame(columns=['parents', 'has_nurs', 'form', 'children', 'housing', 'finance',
       'social', 'health', 'class'])
        while len(fold) < fold_size:
            if i<len(dataset):
                fold=fold.append(dataset.iloc[i])
                i=i+1
            else: break   
        dataset_split[p].append(fold)    
        p=p+1
    if dataset_split[folds]:
        dataset_split[folds-1][0]=dataset_split[folds-1][0].append(dataset_split[folds])
        dataset_split.pop()
    else:    
        dataset_split.pop()
        
    return dataset_split

In [7]:
#Building the algorithm

#Entropy
def entropy(Y):  
    Ent = 0
    values = Y.unique()
    for val in values:
        frac = Y.value_counts()[val] / len(Y)
        Ent = Ent - frac * math.log2(frac)
    return Ent

#Information Gain
def gain(X, Y, att):
    Ent_X = entropy(Y)
    values = X[att].unique()
    Ent_sum = 0
    for val in values:
        index = X.index[X[att] == val].tolist()
        Y_temp = Y.iloc[index]
        Y_temp = Y_temp.reset_index(drop=True)
        frac = len(Y_temp)/len(Y)
        Ent_sum = Ent_sum + frac * entropy(Y_temp)
    return (Ent_X - Ent_sum)

#Deciding attribute for split
def decide_att(X, Y, parent_att):
    attribute = None
    _gain = -100000
    for att in X.keys():
        temp = gain(X, Y, att)
        if temp > _gain:
            if (att in parent_att):
                continue
            _gain = temp
            attribute = att
    if attribute is None:
        return parent_att[-1]
    return attribute

#Sub-Tree
def get_sub_data(X, Y, att, val):
    index = X.index[X[att] == val].tolist()
    X_temp = X.iloc[index, : ]
    Y_temp = Y.iloc[index]
    X_temp = X_temp.reset_index(drop=True)
    Y_temp = Y_temp.reset_index(drop=True)
    return X_temp, Y_temp

In [8]:
#Getting the tree
def get_tree(X, Y, parent_att, count, tree = None):
    current_att = decide_att(X,Y,parent_att)
    values = X[current_att].unique()
    if tree is None:                    
        tree = {}
        tree[current_att] = {}
    for val in values:
        X_sub, Y_sub = get_sub_data(X, Y, current_att, val)
        y_values = Y_sub.unique()
        class_count = {}
        for y_val in y_values:
            class_count[y_val] = Y_sub.value_counts()[y_val]
        maximum = max(class_count, key=class_count.get)
        total = 0
        for i in class_count.values():
            total = total + i
        if (count <= 1):
            tree[current_att][val] = maximum
        elif(class_count[maximum]/total == 1):
            tree[current_att][val] = maximum
        else:
            new_parents = parent_att.copy()
            new_parents.append(current_att)
            tree[current_att][val] = get_tree(X_sub, Y_sub, new_parents, count-1)
    return tree

In [19]:
parent=[]
tree=get_tree(X_train,Y_train,parent,8)

In [20]:
#Printing the tree
def print_tree(dic,level):
    if type(dic)!=dict:
        print(": "+dic)
        return
    for key in dic:
        print()
        val = dic[key]
        if type(val)==dict:
            for k in val:
                for i in range(level-1):
                    print("\t",end="")
                if level!=0 :
                    print("|---->", end="")
                    print("\t",end="")
                print(key+" = "+str(k),end=" ")
                print_tree(val[k],level+1)

In [21]:
print_tree(tree,0)


health = recommended 
|---->	has_nurs = very_crit 
	|---->	housing = convenient 
		|---->	social = problematic 
			|---->	form = complete 
				|---->	children = 3 : spec_prior
				|---->	children = 2 : spec_prior
				|---->	children = 1 : priority
				|---->	children = more : spec_prior
			|---->	form = completed : spec_prior
			|---->	form = incomplete : spec_prior
			|---->	form = foster : spec_prior
		|---->	social = slightly_prob 
			|---->	finance = inconv 
				|---->	children = 2 
					|---->	form = complete : priority
					|---->	form = foster : spec_prior
					|---->	form = completed : priority
					|---->	form = incomplete : spec_prior
				|---->	children = 1 
					|---->	form = complete : priority
					|---->	form = foster : spec_prior
					|---->	form = completed : priority
					|---->	form = incomplete : priority
				|---->	children = more : spec_prior
				|---->	children = 3 : spec_prior
			|---->	finance = convenient : priority
		|---->	social = nonprob 
			|---->	financ

				|---->	children = more : priority
				|---->	children = 3 : priority
				|---->	children = 2 : priority
			|---->	form = incomplete : priority
		|---->	housing = convenient 
			|---->	parents = great_pret : priority
			|---->	parents = usual 
				|---->	finance = inconv 
					|---->	children = 3 : priority
					|---->	children = 1 
						|---->	form = incomplete : very_recom
						|---->	form = complete : very_recom
						|---->	form = completed : very_recom
						|---->	form = foster : priority
					|---->	children = more : priority
					|---->	children = 2 
						|---->	form = completed : very_recom
						|---->	form = foster : priority
				|---->	finance = convenient : very_recom
			|---->	parents = pretentious 
				|---->	finance = convenient : very_recom
				|---->	finance = inconv 
					|---->	children = 3 : priority
					|---->	children = 1 : very_recom
					|---->	children = 2 
						|---->	form = completed : very_recom
						|---->	form = incomplete : priority
					|---->	

					|---->	children = 3 : priority
					|---->	children = 1 : very_recom
				|---->	housing = critical : priority
			|---->	form = complete 
				|---->	children = more 
					|---->	housing = critical : priority
					|---->	housing = convenient : very_recom
					|---->	housing = less_conv : priority
				|---->	children = 1 
					|---->	housing = convenient : recommend
					|---->	housing = critical : very_recom
					|---->	housing = less_conv : very_recom
				|---->	children = 3 : priority
				|---->	children = 2 : very_recom
			|---->	form = incomplete 
				|---->	children = more : priority
				|---->	children = 2 : priority
				|---->	children = 1 : very_recom
				|---->	children = 3 : priority
		|---->	social = slightly_prob 
			|---->	housing = less_conv 
				|---->	children = 1 
					|---->	form = complete : very_recom
					|---->	form = foster : priority
					|---->	form = incomplete : very_recom
					|---->	form = completed : very_recom
				|---->	children = more : priority
				

			|---->	form = complete 
				|---->	children = 2 : spec_prior
				|---->	children = more : spec_prior
				|---->	children = 3 : spec_prior
				|---->	children = 1 : priority
			|---->	form = completed : spec_prior
			|---->	form = incomplete : spec_prior
		|---->	housing = less_conv 
			|---->	children = 3 : spec_prior
			|---->	children = 1 
				|---->	form = completed : priority
				|---->	form = foster : spec_prior
				|---->	form = incomplete : priority
			|---->	children = more : spec_prior
			|---->	children = 2 
				|---->	form = foster : spec_prior
				|---->	form = complete : priority
				|---->	form = completed : priority
				|---->	form = incomplete : spec_prior
		|---->	housing = convenient 
			|---->	finance = convenient : priority
			|---->	finance = inconv 
				|---->	children = 3 : spec_prior
				|---->	children = 1 
					|---->	form = complete : priority
					|---->	form = foster : spec_prior
					|---->	form = incomplete : priority
				|---->	children = more : spec_

In [23]:
#Predicting
def predict1(X,tree,a):
    if type(tree)!=dict:
        a=a.append(tree)
    else:
        for key in tree:
            val=X[key]
            n_tree=tree[key]
            if val in n_tree:
                predict1(X,n_tree[val],a)
            else:
                for i in n_tree:
                    predict1(X,n_tree[i],a)
                    
def predict2(a,s):
    s.append(max(set(a), key = a.count))                    

In [11]:
#Converting the class to numeric attributes
#0:'not_recom',1:'recommend',2:'very_recom',3:'priority',4:'spec_prior'
def convert(list):
    a=[]
    for i in list:
        if i=='not_recom':
            a.append(0)
        elif i=='recommend':
            a.append(1)
        elif i=='very_recom':
            a.append(2)
        elif i=='priority':
            a.append(3)
        else:
            a.append(4)
    return a   

In [12]:
#Confusion Matrix
def confusion_matric(Y_pred,Y_test):
    Y_pred=convert(Y_pred)
    Y_test=convert(Y_test)
    arr = [[0 for p in range(5)] for q in range(5)]
    for i in range(len(Y_pred)):
        arr[Y_pred[i]][Y_test[i]]=arr[Y_pred[i]][Y_test[i]]+1
    return arr

In [13]:
#Precision=TP/(TP+FP)
def precision(a):
    p=0
    for i in range(len(a)):
        n=0
        d=0
        for j in range(len(a)):
            if(i==j):
                n=n+a[i][i]
            d=d+a[i][j]    
        if(d!=0):
            p=p+float(n/d)
        else:
            p=p+0
    p=float(p/len(a))
    return p
                

In [14]:
#Recall=TP/(TP+FN)
def recall(a):
    p=0
    for i in range(len(a)):
        n=0
        d=0
        for j in range(len(a)):
            if(i==j):
                n=n+a[i][i]
            d=d+a[j][i]    
        if(d!=0):
            p=p+float(n/d)
        else:
            p=p+0
    p=float(p/len(a))
    return p

In [15]:
#F1-score=2*precision*recall/(precision+recall)
def F1(a):
    r=recall(a)
    p=precision(a)
    F=float((2*r*p)/(r+p))
    return F

In [16]:
def accuracy(a):
    p=0
    d=0
    n=0
    for i in range(len(a)):
        for j in range(len(a)):
            if(i==j):
                n=n+a[i][i]
            d=d+a[i][j]    
    p=float(n*100/d)
    return p

In [17]:
#cross validation accuracy
def perform(D):
    n=len(D)
    p=0
    for i in range(n):
        test=pd.DataFrame(columns=['parents', 'has_nurs', 'form', 'children', 'housing', 'finance',
       'social', 'health', 'class'])
        train=pd.DataFrame(columns=['parents', 'has_nurs', 'form', 'children', 'housing', 'finance',
       'social', 'health', 'class'])
        test=test.append(D[i][0])
        for j in range(n):
            if(j!=i):
                train=train.append(D[j][0])
        test=test.reset_index(drop=True) 
        train=train.reset_index(drop=True)
        X_train=train.iloc[:,:-1].reset_index(drop=True)
        Y_train=train.iloc[:,-1].reset_index(drop=True)
        X_test=test.iloc[:,:-1].reset_index(drop=True)
        Y_test=test.iloc[:,-1].reset_index(drop=True)
        parents = []
        tree = get_tree(X_train, Y_train, parents, 20, None)
        Y_pred=[]
        for u in range(len(X_test)):
            a=[]
            predict1(X_test.iloc[u,:],tree,a)
            predict2(a,Y_pred)
        cnfmtx=confusion_matric(Y_pred,Y_test)
        acc=accuracy(cnfmtx)
        p=p+acc 
    return p/n    
        
    
                

In [25]:
#prediction results for bootstrap
Y_pred=[]
for i in range(len(X_test)):
    a=[]
    predict1(X_test.iloc[i,:],tree,a)
    predict2(a,Y_pred)
cnf=confusion_matric(Y_pred,Y_test)
acc=accuracy(cnf)
print('Accuracy after bootstrapping: '+str(acc))
prec=precision(cnf)
print('Precision after bootstrapping: '+str(prec))
recll=recall(cnf)
print('Recall after bootstrapping: '+str(recll))
f=F1(cnf)
print('F1-score after bootstrapping: '+str(f))



Accuracy after bootstrapping: 98.31018518518519
Precision after bootstrapping: 0.7610778886996148
Recall after bootstrapping: 0.7586042630462592
F1-score after bootstrapping: 0.7598390626809461


In [50]:
#printing confusion_matrix
def print_cnfmt(a):
    for i in range(len(a)):
        for j in range(len(a)):
            print(a[j][i], end=" | ")
        print("")   

In [49]:
print_cnfmt(cnf)

1473 | 0 | 0 | 0 | 0 | 
0 | 0 | 0 | 0 | 0 | 
0 | 1 | 98 | 19 | 0 | 
0 | 0 | 18 | 1414 | 24 | 
0 | 0 | 0 | 11 | 1262 | 


In [52]:
#prediction results for k-fold cross validation

#5-folds
data=cross_validation_split(X_data,5)
per1=perform(data)
print('Accuracy after 5-folds cross validation: '+str(per1))

#10-folds
data=cross_validation_split(X_data,10)
per2=perform(data)
print('Accuracy after 10-folds cross validation: '+str(per2))


Accuracy after 5-folds cross validation: 98.49537037037038
Accuracy after 10-folds cross validation: 98.85802469135801
